# Plot Viual Odometry

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import copy

def transform(t_x, t_y, t_z, r_x, r_y, r_z, r_w):
    return np.asarray(
        [[1-2*r_y**2-2*r_z**2, 2*r_x*r_y-2*r_z*r_w, 2*r_x*r_z+2*r_y*r_w, t_x],
         [2*r_x*r_y+2*r_z*r_w, 1-2*r_x**2-2*r_z**2, 2*r_y*r_z-2*r_x*r_w, t_y],
         [2*r_x*r_z-2*r_y*r_w, 2*r_y*r_z+2*r_x*r_w, 1-2*r_x**2-2*r_y**2, t_z],
         [0,0,0,1]])
# https://en.wikipedia.org/wiki/Rotation_matrix#General_rotations

In [2]:
df = pd.read_csv("/home/torsten/ms/data/rosbag2_2023_06_21-05_37_55/vo.csv")
df

,timestamp,t_x,t_y,t_z,r_x,r_y,r_z,r_w
0,1687341198,-1.625407e-05,-1.244574e-05,-4.873309e-05,0.000000,0.000000,0.000000,1.0
1,1687341259,2.016457e-07,1.455325e-08,-3.030925e-07,0.000000,0.000000,0.000000,1.0
2,1687340329,-8.859580e-14,2.710610e-13,-5.506706e-14,0.000000,0.000000,0.000000,1.0
3,1687340397,3.330669e-15,5.828671e-16,-2.220446e-15,0.000000,0.000000,0.000000,1.0
4,1687340456,2.627522e-03,1.311804e-03,3.900251e-03,0.000043,-0.000068,-0.000159,1.0
...,...,...,...,...,...,...,...,...
2231,1687341067,2.327449e-07,1.686645e-07,-8.009211e-07,0.000000,0.000000,0.000000,1.0
2232,1687341142,2.199299e-04,-1.129398e-03,-2.040089e-03,-0.000198,-0.000053,-0.000019,1.0
2233,1687341208,7.678047e-05,-7.170402e-05,-1.544256e-04,-0.000009,-0.000008,-0.000006,1.0
2234,1687341264,-1.984008e-08,1.722396e-08,3.684439e-09,0.000000,0.000000,0.000000,1.0


In [3]:
df_kf = pd.read_csv("/home/torsten/ms/data/rosbag2_2023_06_21-05_37_55/kf.csv")
df_kf

,timestamp,t_x,t_y,t_z,r_x,r_y,r_z,r_w
0,1687340945,0.581486,-0.050316,-1.921209,-0.013981,-0.021642,-0.013101,0.999584
1,1687340866,-0.251871,0.107150,1.994880,0.002220,-0.044183,0.012639,0.998942
2,1687340372,0.094818,-0.032737,2.044571,-0.004362,0.007255,-0.006938,0.999941
3,1687340394,0.026764,-0.012133,2.020029,-0.000248,-0.001219,0.005604,0.999984
4,1687341317,-0.001175,0.000103,2.023226,0.002654,0.001491,0.003516,0.999989
5,1687341313,-0.001463,-0.007828,2.053440,-0.000530,0.004001,0.003747,0.999985
6,1687341328,0.015819,0.006905,2.016451,0.004050,-0.002205,0.008922,0.999950
7,1687340981,0.131869,-0.016057,1.995825,-0.001709,0.035250,-0.010601,0.999321
8,1687340598,0.185788,0.009570,0.588236,0.000045,0.129073,-0.001184,0.991635
9,1687341110,0.141943,-0.003480,0.623704,-0.000744,0.129264,0.004024,0.991602


In [4]:
poses = []
poses_kf = []
last_tf = transform(0,0,0,0,0,0,1)
#"""
for i, row in df.iterrows():
    tf = transform(row["t_x"], row["t_y"], row["t_z"], row["r_x"], row["r_y"], row["r_z"], row["r_w"])
    #print(tf)
    #last_tf = np.matmul(tf, last_tf)
    last_tf = np.matmul(last_tf, tf)
    poses.append(copy(last_tf))

last_tf = transform(0,0,0,0,0,0,1)
for i, row in df_kf.iterrows():
    tf = transform(row["t_x"], row["t_y"], row["t_z"], row["r_x"], row["r_y"], row["r_z"], row["r_w"])
    #print(tf)
    #last_tf = np.matmul(tf, last_tf)
    last_tf = np.matmul(last_tf, tf)
    poses_kf.append(copy(last_tf))
#"""

    
print(last_tf)


    

[[ 0.98282929 -0.16837632 -0.07547692  2.92214665]
 [ 0.16944988  0.98550629  0.00800667 -1.83385476]
 [ 0.07303485 -0.02065872  0.99711579 -3.81589575]
 [ 0.          0.          0.          1.        ]]


In [5]:
"""
poses.append(transform(0,0,0,0,0,0,1))

for i in range(3):
    tf =transform(0.2,0,0,0,0.707,0,0.707)
    last_tf = np.matmul(tf, last_tf)
    poses.append(copy(last_tf))

#poses.append(transform(2,0,0,0,0,0,1))
#poses.append(transform(3,0,0,0,0,0,1))
"""


'\nposes.append(transform(0,0,0,0,0,0,1))\n\nfor i in range(3):\n    tf =transform(0.2,0,0,0,0.707,0,0.707)\n    last_tf = np.matmul(tf, last_tf)\n    poses.append(copy(last_tf))\n\n#poses.append(transform(2,0,0,0,0,0,1))\n#poses.append(transform(3,0,0,0,0,0,1))\n'

In [6]:
poses = np.asarray(poses)
poses.shape

(2236, 4, 4)

In [7]:
poses_kf = np.asarray(poses_kf)
poses_kf.shape

(47, 4, 4)

In [8]:
%matplotlib tk
plt.figure(figsize=(18,10))
plt.axis('equal')
plt.plot(poses[:, 0, 3], poses[:,2,3], "-x")
plt.plot(poses_kf[:, 0, 3], poses_kf[:,2,3], "-x")
for i in range(0, poses.shape[0], 100):
    plt.text(poses[i,0,3], poses[i,2,3], str(i))
plt.show()

In [9]:
#df = pd.read_csv("gpsNED.csv")
#df

In [10]:
"""
plt.figure(figsize=(18,10))
plt.scatter(df.iloc[:,0], df.iloc[:,1])
plt.show()
"""

'\nplt.figure(figsize=(18,10))\nplt.scatter(df.iloc[:,0], df.iloc[:,1])\nplt.show()\n'